In [1]:
import pandas as pd
import os

In [2]:
data_path = './q5-dataeng-forecasting-features/input_data/data'
brand_csv = os.path.join(data_path, 'brand.csv')
product_csv = os.path.join(data_path, 'product.csv')
sales_csv = os.path.join(data_path, 'sales.csv')
store_csv = os.path.join(data_path, 'store.csv')

In [16]:
brand = pd.read_csv(brand_csv)
product = pd.read_csv(product_csv)
store = pd.read_csv(store_csv)
sales = pd.read_csv(sales_csv)

In [ ]:
def filter_date_range(df, min_date="2021-01-08", max_date="2021-05-30"):
    # Ensure 'date' is datetime
    df['date'] = pd.to_datetime(df['date'])

    # Filter based on range
    filtered_df = df[(df['date'] >= min_date) & (df['date'] <= max_date)].copy()

    return filtered_df


In [30]:
import pandas as pd

def load_and_merge_data(sales, product, brand, store):
    # Merge product with brand
    product = product.merge(brand, left_on='brand', right_on='name', suffixes=('_product', '_brand'))
    product = product.rename(columns={'id_product': 'product_id', 'id_brand': 'brand_id'})
    
    # Merge sales with product and store
    sales = sales.merge(product[['product_id', 'brand_id']], left_on='product', right_on='product_id')
    sales = sales.merge(store.rename(columns={'id': 'store_id'}), left_on='store', right_on='store_id')

    # Ensure 'date' column is datetime
    sales['date'] = pd.to_datetime(sales['date'])

    return sales

def compute_product_features(df):
    # Group by product_id, store_id, and date to get sales_product
    product_sales = df.groupby(['product_id', 'store_id', 'brand_id', 'date'])['quantity'].sum().reset_index()
    product_sales = product_sales.rename(columns={'quantity': 'sales_product'})

    # Sort for rolling computation
    product_sales = product_sales.sort_values(['product_id', 'store_id', 'date'])

    # Compute MA7_P
    product_sales['MA7_P'] = product_sales.groupby(['product_id', 'store_id'])['sales_product'].transform(
        lambda x: x.rolling(window=7, min_periods=1).mean()
    )

    # Compute LAG7_P
    product_sales['LAG7_P'] = product_sales.groupby(['product_id', 'store_id'])['sales_product'].shift(7)

    return product_sales


In [31]:

# Merge all data
merged_data = load_and_merge_data(sales, product, brand, store)

# Compute product-level features
product_features = compute_product_features(merged_data)

# See the result
print(product_features.head(10))

   product_id  store_id  brand_id       date  sales_product      MA7_P  LAG7_P
0           0         0         0 2021-01-01              6   6.000000     NaN
1           0         0         0 2021-01-02             19  12.500000     NaN
2           0         0         0 2021-01-03             14  13.000000     NaN
3           0         0         0 2021-01-04             10  12.250000     NaN
4           0         0         0 2021-01-05              7  11.200000     NaN
5           0         0         0 2021-01-06              6  10.333333     NaN
6           0         0         0 2021-01-07             18  11.428571     NaN
7           0         0         0 2021-01-08             10  12.000000     6.0
8           0         0         0 2021-01-09             10  10.714286    19.0
9           0         0         0 2021-01-10              3   9.142857    14.0


In [32]:
product_features.columns

Index(['product_id', 'store_id', 'brand_id', 'date', 'sales_product', 'MA7_P',
       'LAG7_P'],
      dtype='object')

In [33]:
def compute_brand_features(df):
    # Group by brand_id, store_id, and date to get sales_brand
    brand_sales = df.groupby(['brand_id', 'store_id', 'date'])['quantity'].sum().reset_index()
    brand_sales = brand_sales.rename(columns={'quantity': 'sales_brand'})

    # Sort for rolling
    brand_sales = brand_sales.sort_values(['brand_id', 'store_id', 'date'])

    # Compute MA7_B
    brand_sales['MA7_B'] = brand_sales.groupby(['brand_id', 'store_id'])['sales_brand'].transform(
        lambda x: x.rolling(window=7, min_periods=1).mean()
    )

    # Compute LAG7_B
    brand_sales['LAG7_B'] = brand_sales.groupby(['brand_id', 'store_id'])['sales_brand'].shift(7)

    return brand_sales


In [34]:
def compute_store_features(df):
    # Group by store_id and date to get total store sales
    store_sales = df.groupby(['store_id', 'date'])['quantity'].sum().reset_index()
    store_sales = store_sales.rename(columns={'quantity': 'sales_store'})

    # Sort for rolling
    store_sales = store_sales.sort_values(['store_id', 'date'])

    # Compute MA7_S
    store_sales['MA7_S'] = store_sales.groupby('store_id')['sales_store'].transform(
        lambda x: x.rolling(window=7, min_periods=1).mean()
    )

    # Compute LAG7_S
    store_sales['LAG7_S'] = store_sales.groupby('store_id')['sales_store'].shift(7)

    return store_sales


In [35]:
def merge_all_features(product_feat, brand_feat, store_feat):
    # Merge on product_id, store_id, date
    df = product_feat.merge(brand_feat, on=['store_id', 'brand_id', 'date'], how='left')
    df = df.merge(store_feat, on=['store_id', 'date'], how='left')
    df = df.sort_values(by=['product_id', 'brand_id', 'store_id', 'date']).reset_index(drop=True)

    return df


In [36]:
df = merge_all_features(product_features, compute_brand_features(merged_data), compute_store_features(merged_data))

In [37]:
df.to_csv("features.csv", index=False)


,product_id,store_id,brand_id,date,sales_product,MA7_P,LAG7_P,sales_brand,MA7_B,LAG7_B,sales_store,MA7_S,LAG7_S
0,0,0,0,2021-01-01,6,6.000000,NaN,21,21.000000,NaN,39,39.000000,NaN
1,0,0,0,2021-01-02,19,12.500000,NaN,29,25.000000,NaN,48,43.500000,NaN
2,0,0,0,2021-01-03,14,13.000000,NaN,34,28.000000,NaN,53,46.666667,NaN
3,0,0,0,2021-01-04,10,12.250000,NaN,12,24.000000,NaN,29,42.250000,NaN
4,0,0,0,2021-01-05,7,11.200000,NaN,40,27.200000,NaN,57,45.200000,NaN
5,0,0,0,2021-01-06,6,10.333333,NaN,18,25.666667,NaN,32,43.000000,NaN
6,0,0,0,2021-01-07,18,11.428571,NaN,46,28.571429,NaN,64,46.000000,NaN
7,0,0,0,2021-01-08,10,12.000000,6.0,40,31.285714,21.0,63,49.428571,39.0
8,0,0,0,2021-01-09,10,10.714286,19.0,24,30.571429,29.0,41,48.428571,48.0
9,0,0,0,2021-01-10,3,9.142857,14.0,26,29.428571,34.0,52,48.285714,53.0


In [41]:
def compute_top_wmape(df, top=10):
    # Drop rows with missing data
    df_valid = df.dropna(subset=['sales_product', 'MA7_P'])

    # Calculate WMAPE
    wmape_df = (
        df_valid
        .groupby(['product_id', 'store_id', 'brand_id'])
        .apply(lambda g: (abs(g['sales_product'] - g['MA7_P']).sum()) / g['sales_product'].sum()
               if g['sales_product'].sum() != 0 else None)
        .reset_index(name='WMAPE')
    )

    # Sort and take top N
    top_wmape_df = wmape_df.sort_values(by='WMAPE', ascending=False).head(top)

    return top_wmape_df


In [42]:
wmape_result = compute_top_wmape(df)


/var/folders/20/rp4q37kj26x7ymknfx9m892r0000gn/T/ipykernel_22573/688832240.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: (abs(g['sales_product'] - g['MA7_P']).sum()) / g['sales_product'].sum()


In [44]:
wmape_result.head(11)

,product_id,store_id,brand_id,WMAPE
5,1,0,0,0.608102
17,3,2,0,0.531721
20,4,0,1,0.529127
1,0,1,0,0.517529
13,2,3,1,0.515781
10,2,0,1,0.512986
11,2,1,1,0.512819
22,4,2,1,0.512040
19,3,4,0,0.496040
21,4,1,1,0.490999


In [ ]:
wmape_result.to_csv("mapes.csv", index=False)
